In [1]:
import numpy as np
import csv

In [2]:
# which feature you want to extract
feature_set=[7,8,9,10]

In [3]:
# Predict the next hour before using n hours（ upper bound = 9 ）
prev_hrs=9

In [4]:
iteration=100000

In [5]:
raw = []
with open("train.csv",'r',encoding="big5") as train_f:
    train_f.readline() #ignore the 1st line
    for line in train_f:
        line = line.replace('NR', '0.0')
        data = line.strip('\n').split(',')
        data = data[3:]
        data = list(map(float, data))
        raw.append(data)

In [6]:
# Reshape as monthly data
train_data = []
for m in range(12):
    month_data = []

    for f in range(18):
        tmp = []

        for d in range(20):
            tmp += raw[360*m+18*d+f]

        month_data.append(tmp)

    train_data.append(month_data)

In [7]:
# Feature extraction
x = []
y_hat = [] #ground truth
for m in range(12):
    for hr in range(480-prev_hrs):
        y_hat.append(train_data[m][9][hr+prev_hrs])
        x.append([])
        for f in range(18):
            # Pick features that we want
            if f in feature_set:
                x[(480-prev_hrs)*m+hr] += train_data[m][f][hr:hr+prev_hrs]

In [8]:
x = np.array(x)
y_hat = np.array(y_hat)

print(x.shape)
print(y_hat.shape)

(5652, 36)
(5652,)


In [15]:
# Regression with gradient descent
# dim(x)=len(x) by (prev_hrs*# features)
# dim(y_hat)=len(x)

# y = b + sum(w_Tx)
# Randomly determine model parameters
b = 1
w = np.random.randn(len(x[0]))
###################################
# you must set your learning rate #
###################################
lr = 1
ada_b = 0.0
ada_w = np.zeros(len(w))

prev_loss = float('inf')

# Gradient descent with adagrad
for k in range(iteration):
    b_grad = 0.0
    w_grad = np.zeros(len(w))

    # Set loss function as square error
    # Predicted value y from nth data = nth entry of X*w+b
    y = np.dot(x,w) + b
    dy = y_hat - y

    # Check current loss
    ###########################
    # you must implement here #
    ###########################
    loss = (np.sum(np.square(dy)))/len(y)
    if(k%100==0):
        print('%d iteration with loss %f\n' % (k, loss))

    # Stop when little improvement
    if(np.abs(prev_loss-loss) <= 1e-8):
        break

    # Compute gradients
    b_grad -= np.sum(2*dy)
    
    for i in range(len(w)):
        ###########################
        # you must implement here #
        ###########################
        w_grad[i] = w_grad[i]- np.dot(2*dy, x.T[i])

    # Compute AdaGrad terms
    ada_b += b_grad ** 2
    ada_w += w_grad ** 2

    # Update W, b
    b = b - lr/np.sqrt(ada_b) * b_grad
    for i in range(len(w)):
        ###########################
        # you must implement here #
        ###########################
        w[i] = w[i] - ( lr/np.sqrt(ada_w[i])) * w_grad[i]
        
    prev_loss = loss

0 iteration with loss 63991.899412

100 iteration with loss 669.957832

200 iteration with loss 294.440270

300 iteration with loss 184.011811

400 iteration with loss 137.158327

500 iteration with loss 112.480567

600 iteration with loss 97.244193

700 iteration with loss 86.694777

800 iteration with loss 78.812921

900 iteration with loss 72.633344

1000 iteration with loss 67.638187

1100 iteration with loss 63.518459

1200 iteration with loss 60.072635

1300 iteration with loss 57.160023

1400 iteration with loss 54.677467

1500 iteration with loss 52.546682

1600 iteration with loss 50.706790

1700 iteration with loss 49.109591

1800 iteration with loss 47.716401

1900 iteration with loss 46.495830

2000 iteration with loss 45.422174

2100 iteration with loss 44.474223

2200 iteration with loss 43.634349

2300 iteration with loss 42.887814

2400 iteration with loss 42.222225

2500 iteration with loss 41.627107

2600 iteration with loss 41.093561

2700 iteration with loss 40.6139

22600 iteration with loss 35.216084

22700 iteration with loss 35.216066

22800 iteration with loss 35.216049

22900 iteration with loss 35.216032

23000 iteration with loss 35.216016

23100 iteration with loss 35.216000

23200 iteration with loss 35.215985

23300 iteration with loss 35.215971

23400 iteration with loss 35.215957

23500 iteration with loss 35.215943

23600 iteration with loss 35.215930

23700 iteration with loss 35.215917

23800 iteration with loss 35.215905

23900 iteration with loss 35.215894

24000 iteration with loss 35.215882

24100 iteration with loss 35.215871

24200 iteration with loss 35.215861

24300 iteration with loss 35.215851

24400 iteration with loss 35.215841

24500 iteration with loss 35.215831

24600 iteration with loss 35.215822

24700 iteration with loss 35.215813

24800 iteration with loss 35.215805

24900 iteration with loss 35.215796

25000 iteration with loss 35.215788

25100 iteration with loss 35.215781

25200 iteration with loss 35.215773

2

In [16]:
test_raw = []
with open("test.csv", 'r') as test_f:
    for line in test_f:
        line = line.replace('NR', '0.0')
        data = line.strip('\n').split(',')
        test_raw.append(data)

In [17]:
output_csv = []
print('id,value')
for i in range(240):
    idx = test_raw[i*18][0]

    test_x = []
    for f in range(18):
        if f in feature_set:
            test_x += test_raw[i*18+f][2+(9-prev_hrs):11]

    test_x = list(map(float, test_x))
    test_x = np.array(test_x)

    y = np.dot(w,test_x)+b

    print('%s,%f' % (idx, y))
    output_csv.append([idx,y])

id,value
id_0,25.362313
id_1,64.240498
id_2,20.425071
id_3,29.040841
id_4,9.986065
id_5,36.252008
id_6,39.242771
id_7,16.184686
id_8,51.099673
id_9,33.385746
id_10,38.994594
id_11,55.265114
id_12,35.113041
id_13,39.657852
id_14,26.919467
id_15,12.478107
id_16,51.593310
id_17,22.904708
id_18,17.383637
id_19,18.473782
id_20,15.734968
id_21,30.449184
id_22,38.093639
id_23,9.575847
id_24,58.442425
id_25,44.058593
id_26,26.674381
id_27,8.593740
id_28,30.781184
id_29,35.799236
id_30,17.243511
id_31,11.271773
id_32,16.435219
id_33,6.780919
id_34,16.952694
id_35,40.117124
id_36,19.964537
id_37,27.131571
id_38,10.221067
id_39,62.158532
id_40,47.447787
id_41,18.696058
id_42,52.276474
id_43,18.212250
id_44,44.742535
id_45,50.409227
id_46,59.861193
id_47,41.713278
id_48,50.882381
id_49,39.988328
id_50,28.697248
id_51,23.517607
id_52,40.138400
id_53,24.104890
id_54,38.082099
id_55,41.183358
id_56,24.337938
id_57,22.782654
id_58,36.746924
id_59,32.523355
id_60,15.345420
id_61,24.760166
id_62,32.2030

In [18]:
fileHeader = ['id','value']
csvFile = open('answer.csv', "w+")
writer = csv.writer(csvFile)
writer.writerow(fileHeader)
for j in range(len(output_csv)):
    writer.writerow(output_csv[j])
csvFile.close()